In [916]:
import pandas as pd
import numpy as np

In [917]:
df = pd.read_excel('criminal_df_v1.xlsx')

In [1041]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2723 entries, 0 to 2722
Data columns (total 12 columns):
Applicant         2722 non-null object
Defendant         2720 non-null object
Case No           2723 non-null object
Judges            2723 non-null object
Summary           2723 non-null object
Date Heard        2072 non-null object
Date Judgement    2318 non-null object
Court             2723 non-null object
Case Category     2723 non-null object
Province          2723 non-null object
Related Crimes    1470 non-null object
Charge            2723 non-null object
dtypes: object(12)
memory usage: 255.4+ KB


In [919]:
# to replace '\\'96'
# to add bail, compensation amounts

In [920]:
df['Charge'] = df['Charge'].replace('theft', 'robbery')
df['Charge'] = df['Charge'].replace('prescribed sentences', 'prescribed sentence')
df['Charge'] = df['Charge'].replace('prescribed minimum sentences', 'prescribed sentence')
df['Charge'] = df['Charge'].replace('housebreaking with intent to steal and theft', 'housebreaking')

In [921]:
df['Charge'].value_counts()[:10]

evidence               396
rape                   275
arrest                 196
murder                 177
appeal                 171
review                 127
prescribed sentence    118
robbery                110
bail                    91
damages                 56
Name: Charge, dtype: int64

In [922]:
# lack of information

# remit
# dismiss
# inadmissible, common law position restored
# set aside
# appeal granted, appeal upheld

# life sentence (25 years) and amount
# prison sentence and amount
# bail and amount

In [923]:
from textblob import TextBlob
import nltk
lemma = nltk.wordnet.WordNetLemmatizer()

def lemma_words(text):
    
    # returns clean text with infinitive verbs

    blob = TextBlob(text)
    blob_words = list(blob.words)
    
    blob_lst = []

    for word in blob_words:
        
        new_word = lemma.lemmatize(word, 'v')

        blob_lst.append(new_word)
        
    return ' '.join(blob_lst)

In [1030]:
def get_sentence_years(blob):

    sentence_lst = []

    # extract ngrams that include keywords like sentence, imprisonment and not old and age

    for lst in blob.ngrams(3):
        if 'years' in lst and ('old' not in lst and 'age' not in lst):
            sentence_lst.append(list(lst))

    concatted = []

    for item in sentence_lst:     
        concatted.append(' '.join(item))

    concatted = ' '.join(concatted)
    sentence_phrase = list(dict.fromkeys(concatted.split()))

    # append ints that are mentioned in sentence_phrase 

    amount = []

    for word in sentence_phrase:
        try:
            amount.append(int(word))
        except ValueError:
            pass
    return amount

In [951]:
def get_rand_amount(blob):
    

    amount_lst = []

    # extract ngrams that include keywords like sentence, imprisonment and not old and age

    for lst in blob.ngrams(3):
        if 'years' in lst and ('old' not in lst and 'age' not in lst):
            amount_lst.append(list(lst))

    concatted = []

    for item in sentence_lst:     
        concatted.append(' '.join(item))

    concatted = ' '.join(concatted)
    sentence_phrase = list(dict.fromkeys(concatted.split()))

    # append ints that are mentioned in sentence_phrase 

    amount = []

    for word in sentence_phrase:
        try:
            amount.append(int(word))
        except ValueError:
            pass
    return amount

In [1087]:
def get_verdict(text):
    
    text = text.replace('life sentence', '25 years sentence')
    text = text.replace('sentenced to life', '25 years sentence')
    text = text.replace('life imprisonment', '25 years sentence')
    text = text.replace('life in prison', '25 years sentence')
    
    text = lemma_words(text)
    
    blob = TextBlob(text)
    
    blob_lst = blob.ngrams(4)
    
    try:
        
        amount = get_sentence_years(blob)
        
        for item in blob_lst:

            if 'remit' in item:
                verdict = 'Remitted'

            elif 'inadmissible' in item or ('law' in item and 'position' in item and 'restored' in item):
                verdict = 'Inadmissible'

            elif 'set' in item and 'aside' in item:
                verdict = 'Set Aside'

            elif 'uphold' in item and 'grant' in item:
                verdict = 'Appeal Granted'

            elif 'uphold' in item and 'appeal' in item:
                verdict = 'Appeal Upheld'

            elif 'appeal' in item and 'dismiss' in item:
                verdict = 'Appeal Dismissed'

            elif 'award' in item or 'compensate' in item or 'compensation' in item:
                verdict = 'Compensation'

            elif 'suspend' in item and ('court' in item or 'sentence' in item):
                verdict = 'Suspended'
                
            elif 'sentence' in item or 'imprisonment' in item:
                verdict = 'Sentenced'
                
        return verdict, amount
    
    except UnboundLocalError:
        return np.nan, amount

In [1115]:
get_verdict(df['Summary'][50])

('Compensation', [])

In [1220]:
for words in text_test.split(' '):
    try:
        result =  re.search("^r+", words) and re.search("\d+", words)
        print(result.group())
    except AttributeError:
        pass

50
30


In [1090]:
df['Verdict'] = df['Summary'].apply(get_verdict)

In [1091]:
df

,Applicant,Defendant,Case No,Judges,Summary,Date Heard,Date Judgement,Court,Case Category,Province,Related Crimes,Charge,Verdict
0,State,Zibi 2008 JDR 0880 (BHC),CA&R 1/97,"Ebrahim J, Pickard J",criminal law. appeal. contravening s 11(1) of ...,NaN,1997-05-02,Bhisho High Court,criminal law,Eastern Cape,NaN,appeal,"(Remitted, [])"
1,State,Nkosi 2016 JDR 0568 (GJ),CC98/2012,DSS Moshidi J,criminal law. sentence. prescribed minimum sen...,2016-02-26,2016-02-26,"Circuit Local Division, Middelburg",criminal law,Mpumalanga,"Murder, Attempted murder",prescribed sentence,"(Sentenced, [25])"
2,State,Nkosi 2016 JDR 0567 (GP),CC98/2012,DSS Moshidi J,criminal law. murder. attempted murder. houseb...,NaN,2015-12-02,"Circuit Local Division, Middelburg",criminal law,Mpumalanga,"Murder, Attempted murder, Robbery",murder,"(nan, [])"
3,Savoi,National Director of Public Prosecutions 2014 ...,CCT 71/13[2014] ZACC 5,Moseneke ACJ and Skweyiya ADCJ and Cameron J a...,criminal law. prevention of crime. offences. c...,2013-11-11,2014-03-20,Constitutional Court,criminal law,South Africa,Racketeering,prevention of crime,"(nan, [])"
4,State,Schoombee 2017 JDR 0054 (CC) \'97 Now reported...,CCT 154/16[2016] ZACC 50,"Nkabinde ACJ and Cameron J, Froneman J, Jafta ...","criminal law. trial. record. lost, destroyed o...",2016-12-15,2016-12-15,Constitutional Court,criminal law,South Africa,NaN,record,"(Appeal Dismissed, [])"
5,State,Phakane 2017 JDR 1939 (CC) \'97 Now reported a...,CCT 61/16,"Nkabinde ADCJ and Cameron J, Froneman J, Jafta...","criminal law. appeal. record. lost, destroyed ...",2017-12-05,2017-12-05,Constitutional Court,criminal law,South Africa,NaN,appeal,"(Set Aside, [])"
6,State,Okah 2018 JDR 0228 (CC) \'97 Now reported at 2...,CCT 315/16,"Zondo ACJ and Cameron J, Froneman J, Jafta J a...",criminal law. jurisdiction. extraterritorial j...,2017-11-28,2017-02-23,Constitutional Court,criminal law,South Africa,NaN,jurisdiction,"(nan, [])"
7,State,Ndlovu 2017 JDR 1076 (CC) \'97 Now reported at...,CCT 174/16,"Nkabinde ADCJ and Cameron J, Froneman J, Jafta...",criminal law. rape. sentence. life imprisonmen...,2017-02-23,2017-06-15,Constitutional Court,criminal law,South Africa,Rape,rape,"(Sentenced, [25])"
8,State,MT 2018 JDR 1469 (CC),CCT 122/17,Mogoeng CJ and Cachalia AJ and Dlodlo AJ and F...,criminal law. prescribed minimum sentences. cr...,2018-05-10,2018-09-03,Constitutional Court,criminal law,South Africa,NaN,prescribed sentence,"(Sentenced, [])"
9,State,Molaudzi 2015 JDR 1315 (CC),CCT 42/15,"Mogoeng CJ and Moseneke DCJ and Cameron J, Fro...",criminal law. court. constitutional court. pow...,NaN,2015-06-25,Constitutional Court,criminal law,South Africa,NaN,court,"(Sentenced, [])"


In [1097]:
df[df.index == 2721]

,Applicant,Defendant,Case No,Judges,Summary,Date Heard,Date Judgement,Court,Case Category,Province,Related Crimes,Charge,Verdict
2721,State,Rossouw 2013 JDR 2572 (WCC) \'97 Now reported ...,A 374/13,Davis J and Schippers J,criminal law. arms and ammunition. unlawful po...,2013-11-08,2013-11-08,"Western Cape High Court, Cape Town",criminal law,Western Cape,NaN,arms and ammunition,"(Appeal Dismissed, [25])"


In [1099]:
df['Summary'][2713]

'criminal law. murder. mens rea. intention to kill. dolus eventualis. appellant working at club as dj. fight breaking out. deceased and friends fleeing but chased by group. incurring fatal shot to head. appellant admitting firing shots but alleging to establish control. denying in direction of deceased. on appeal, court finding not credible that did not fire on moving group or deceased could not have been shot. any righthinking person would have realised could injure or kill someone in so doing. only reasonable inference that reconciled himself with this possibility. murder conviction on basis of dolus eventualis confirmed.'